In [ ]:
import pandas as pd
import numpy as np

dca_data = pd.read_csv("data/elektronisk-rapportering-ers-2018-fangstmelding-dca.csv", sep=";", decimal=",")

In [ ]:
# Keep given columns
id_columns = ["Melding ID", "Starttidspunkt", "Stopptidspunkt"]
# keep_columns = ["Melding ID", "Meldingstidspunkt", "Meldingsnummer", "Meldingsversjon", "Redskapsspesifikasjon", "Starttidspunkt", "Stopptidspunkt", "Radiokallesignal (ERS)", "Varighet",
keep_columns = ["Melding ID", "Meldingstidspunkt", "Starttidspunkt", "Stopptidspunkt", "Radiokallesignal (ERS)", "Varighet",
                "Startposisjon bredde", "Startposisjon lengde", "Havdybde start", "Stopposisjon bredde",
                "Stopposisjon lengde", "Havdybde stopp", "Trekkavstand", "Redskap FAO (kode)", "Hovedart FAO",
                "Art FAO", "Rundvekt", "Bruttotonnasje 1969", "Bruttotonnasje annen",
                "Bredde", "Fartøylengde", "Hovedområde start (kode)", "Hovedområde stopp (kode)"]

reduced_data = dca_data[keep_columns]

In [ ]:
# Keep only OTB (bottom trawl) and drop rows with no species information
reduced_data = reduced_data.where(reduced_data["Redskap FAO (kode)"] == "OTB")
reduced_data = reduced_data.dropna(subset=["Art FAO"])

In [ ]:
reduced_data

In [ ]:
# Sum the round weights for message id, start time, and stop time
catch_sums = reduced_data.groupby(['Melding ID','Starttidspunkt','Stopptidspunkt'])['Rundvekt'].sum()
catch_sums

In [ ]:
# Check for duplicates
reduced_data.duplicated(["Melding ID", "Starttidspunkt", "Stopptidspunkt", "Art FAO"]).sum()

In [ ]:
# Create columns of round weight for each of 14 fish species + column for rest
top_species = ['Torsk', 'Sei', 'Hyse', 'Uer (vanlig)', 'Dypvannsreke', 'Lange', 'Snabeluer', 'Blåkveite', 'Flekksteinbit', 'Lysing', 'Gråsteinbit', 'Breiflabb', 'Kveite', 'Lyr']
reduced_data = reduced_data.loc[reduced_data["Art FAO"].isin(top_species)]
reduced_data_pivot = reduced_data.pivot(index=["Melding ID", "Starttidspunkt", "Stopptidspunkt"], columns="Art FAO", values="Rundvekt").reset_index()
reduced_data_weight = reduced_data_pivot.merge(catch_sums, on=["Melding ID", "Starttidspunkt", "Stopptidspunkt"])

reduced_data_weight["ANDRE"] = reduced_data_weight.apply(lambda row: row["Rundvekt"] - row[top_species].sum(), axis=1)
reduced_data_weight[top_species] = reduced_data_weight[top_species].replace(np.nan, 0)
reduced_data_weight.head()

In [ ]:
reduced_data = reduced_data.drop(columns=["Art FAO", "Rundvekt"]).drop_duplicates()

In [ ]:
# Merge datasets and combine tonnage columns
complete_data = reduced_data.merge(reduced_data_weight, on=["Melding ID", "Starttidspunkt", "Stopptidspunkt"])
complete_data[["Bruttotonnasje 1969", "Bruttotonnasje annen"]] = complete_data[["Bruttotonnasje 1969", "Bruttotonnasje annen"]].replace(np.nan, 0)
complete_data["Bruttotonnasje"] = complete_data.apply(lambda row: row["Bruttotonnasje 1969"] + row["Bruttotonnasje annen"], axis=1)
complete_data.drop(columns=["Bruttotonnasje 1969", "Bruttotonnasje annen"], inplace=True)

In [ ]:
complete_data = complete_data.sort_values(["Meldingstidspunkt", "Starttidspunkt"], ignore_index=True)

# Check for time overlap

In [ ]:
message_ids = complete_data["Melding ID"].unique()
call_signs = complete_data["Radiokallesignal (ERS)"].unique()
print(call_signs)
complete_data["Starttidspunkt"] = pd.to_datetime(complete_data["Starttidspunkt"], format="mixed")
complete_data["Stopptidspunkt"] = pd.to_datetime(complete_data["Stopptidspunkt"], format="mixed")

# Drop time overlapping messages for each vessel
all_messages = []
for c_sign in call_signs:
    messages = complete_data.where(complete_data["Radiokallesignal (ERS)"]==c_sign).dropna(how="all")
    i = 0
    len_df = len(messages)
    while i < len_df-1:
        # if (messages.iloc[i+1]["Melding ID"] == messages.iloc[i]["Melding ID"] and
        # Message ID can be same or different
        if (messages.iloc[i+1]["Starttidspunkt"] < messages.iloc[i]["Stopptidspunkt"] and
            messages.iloc[i+1]["Starttidspunkt"] >= messages.iloc[i]["Starttidspunkt"]):
            # print(f"Overlap between: {messages.index[i]} and {messages.index[i+1]}")
            messages = messages.drop(messages.index[i+1], inplace=False)
            len_df -= 1
        i += 1
    all_messages.append(messages)

complete_data_no_dupes = pd.concat(all_messages)

In [ ]:
print(len(complete_data), len(complete_data_no_dupes))

In [ ]:
complete_data_no_dupes["Trekkavstand"] = complete_data_no_dupes["Trekkavstand"].replace(np.nan, 0)

# Drop rows where area is nan
print(complete_data_no_dupes.isna().any(axis=1).sum())
complete_data_no_dupes = complete_data_no_dupes.dropna()
print(complete_data_no_dupes.isna().any(axis=1).sum())

In [ ]:
# complete_data_no_dupes.to_csv("final.csv", index=False)

# Formatting

In [ ]:
df = complete_data_no_dupes
df = df.sort_values("Starttidspunkt")
df["Meldingstidspunkt"] = pd.to_datetime(df["Meldingstidspunkt"], format="mixed")
df

# Combing data

In [1]:
from main import process_ers_data
import pandas as pd
import os

In [2]:
process_before = []
process_after = []
# file_list = ["elektronisk-rapportering-ers-2018-fangstmelding-dca.csv", "elektronisk-rapportering-ers-2019-fangstmelding-dca.csv"]
for dca_file in os.listdir("data"):
# for dca_file in file_list:
    if dca_file.endswith(".csv"):
        df = pd.read_csv(os.path.join("data", dca_file), sep=";", decimal=",")
        process_after.append(df)
        df_proc = process_ers_data(df)
        process_before.append(df_proc)
                
# print(os.listdir("data"), )

/tmp/ipykernel_54840/3330163636.py:7: DtypeWarning: Columns (51,83,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("data", dca_file), sep=";", decimal=",")
/tmp/ipykernel_54840/3330163636.py:7: DtypeWarning: Columns (51,83,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("data", dca_file), sep=";", decimal=",")
/tmp/ipykernel_54840/3330163636.py:7: DtypeWarning: Columns (51,83,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("data", dca_file), sep=";", decimal=",")
/tmp/ipykernel_54840/3330163636.py:7: DtypeWarning: Columns (51,83,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("data", dca_file), sep=";", decimal=",")
/tmp/ipykernel_54840/3330163636.py:7: DtypeWarning: Columns (51,77,78,83,90) have mixed types. Specify dtype option on import or

In [3]:
pr_bef = pd.concat(process_before)
pr_bef.sort_values("Starttidspunkt")

,Melding ID,Meldingstidspunkt,Starttidspunkt,Stopptidspunkt,Radiokallesignal (ERS),Varighet,Startposisjon bredde,Startposisjon lengde,Havdybde start,Stopposisjon bredde,...,Lange,Lyr,Lysing,Sei,Snabeluer,Torsk,Uer (vanlig),Rundvekt,ANDRE,Bruttotonnasje
4,37626.0,2011-01-01 22:31:00,2010-12-30 19:54:00,2010-12-30 21:10:00,LILQ,76.0,70.827,17.771,-197.0,70.831,...,0.0,0.0,0.0,481.0,0.0,390.0,0.0,871.0,0.0,840.0
0,37565.0,2011-01-01 07:40:00,2010-12-30 21:24:00,2010-12-31 06:21:00,LLDM,537.0,66.104,-30.768,-466.0,66.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,2350.0
5,37628.0,2011-01-01 22:31:00,2010-12-30 22:04:00,2010-12-31 01:01:00,LILQ,177.0,70.823,17.894,-189.0,70.819,...,0.0,0.0,0.0,963.0,0.0,780.0,0.0,1814.0,0.0,840.0
1362,37687.0,2011-01-02 02:31:00,2010-12-30 22:04:00,2010-12-31 01:01:00,LILQ,177.0,70.823,17.894,-189.0,70.819,...,0.0,0.0,0.0,963.0,0.0,780.0,0.0,1814.0,0.0,840.0
6,37628.0,2011-01-01 22:31:00,2010-12-31 07:30:00,2010-12-31 11:46:00,LILQ,256.0,70.469,17.669,-173.0,70.427,...,0.0,0.0,0.0,447.0,0.0,9141.0,0.0,9873.0,0.0,840.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,3302619.0,2024-06-03 23:59:00,2024-06-03 15:39:00,2024-06-03 21:23:00,LFBW,344.0,70.946,20.223,0.0,71.104,...,0.0,0.0,0.0,6930.0,0.0,0.0,624.0,7554.0,0.0,1648.0
1619,3302700.0,2024-06-04 02:50:00,2024-06-03 16:16:00,2024-06-03 23:31:00,LDBR,435.0,62.140,0.082,0.0,62.161,...,0.0,0.0,0.0,0.0,195.0,0.0,0.0,3981.0,0.0,3104.0
1616,3302662.0,2024-06-04 01:47:00,2024-06-03 17:29:00,2024-06-03 23:20:00,LCMP,351.0,72.042,16.056,0.0,72.094,...,0.0,0.0,0.0,508.0,1940.0,282.0,54.0,3233.0,0.0,1853.0
1231,3302569.0,2024-06-03 23:30:00,2024-06-03 19:00:00,2024-06-03 20:34:00,LHXV,94.0,73.250,23.365,0.0,73.163,...,0.0,0.0,0.0,0.0,0.0,1400.0,0.0,1480.0,0.0,2053.0


In [4]:
pr_aft = pd.concat(process_after)
pr_aft = process_ers_data(pr_aft)
pr_aft.sort_values("Starttidspunkt")

,Melding ID,Meldingstidspunkt,Starttidspunkt,Stopptidspunkt,Radiokallesignal (ERS),Varighet,Startposisjon bredde,Startposisjon lengde,Havdybde start,Stopposisjon bredde,...,Lange,Lyr,Lysing,Sei,Snabeluer,Torsk,Uer (vanlig),Rundvekt,ANDRE,Bruttotonnasje
4,37626.0,2011-01-01 22:31:00,2010-12-30 19:54:00,2010-12-30 21:10:00,LILQ,76.0,70.827,17.771,-197.0,70.831,...,0.0,0.0,0.0,481.0,0.0,390.0,0.0,871.0,0.0,840.0
0,37565.0,2011-01-01 07:40:00,2010-12-30 21:24:00,2010-12-31 06:21:00,LLDM,537.0,66.104,-30.768,-466.0,66.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,2350.0
5,37628.0,2011-01-01 22:31:00,2010-12-30 22:04:00,2010-12-31 01:01:00,LILQ,177.0,70.823,17.894,-189.0,70.819,...,0.0,0.0,0.0,963.0,0.0,780.0,0.0,1814.0,0.0,840.0
15571,37687.0,2011-01-02 02:31:00,2010-12-30 22:04:00,2010-12-31 01:01:00,LILQ,177.0,70.823,17.894,-189.0,70.819,...,0.0,0.0,0.0,963.0,0.0,780.0,0.0,1814.0,0.0,840.0
6,37628.0,2011-01-01 22:31:00,2010-12-31 07:30:00,2010-12-31 11:46:00,LILQ,256.0,70.469,17.669,-173.0,70.427,...,0.0,0.0,0.0,447.0,0.0,9141.0,0.0,9873.0,0.0,840.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39319,3302619.0,2024-06-03 23:59:00,2024-06-03 15:39:00,2024-06-03 21:23:00,LFBW,344.0,70.946,20.223,0.0,71.104,...,0.0,0.0,0.0,6930.0,0.0,0.0,624.0,7554.0,0.0,1648.0
55540,3302700.0,2024-06-04 02:50:00,2024-06-03 16:16:00,2024-06-03 23:31:00,LDBR,435.0,62.140,0.082,0.0,62.161,...,0.0,0.0,0.0,0.0,195.0,0.0,0.0,3981.0,0.0,3104.0
55537,3302662.0,2024-06-04 01:47:00,2024-06-03 17:29:00,2024-06-03 23:20:00,LCMP,351.0,72.042,16.056,0.0,72.094,...,0.0,0.0,0.0,508.0,1940.0,282.0,54.0,3233.0,0.0,1853.0
39305,3302569.0,2024-06-03 23:30:00,2024-06-03 19:00:00,2024-06-03 20:34:00,LHXV,94.0,73.250,23.365,0.0,73.163,...,0.0,0.0,0.0,0.0,0.0,1400.0,0.0,1480.0,0.0,2053.0


In [22]:
# Group by month, either by message time or start time
# group_by_month = df.groupby(pd.Grouper(key="Starttidspunkt", freq="ME"))
group_by_month = pr_aft.groupby(pd.Grouper(key="Meldingstidspunkt", freq="ME"))

df_by_month = [month for _, month in group_by_month]
for month in df_by_month:
    month.index = pd.DatetimeIndex(month["Meldingstidspunkt"])

for month_n in df_by_month:
    year = month_n.index.year[0]
    month = month_n.index.month[0]
    print(f"Year: {year}, Month: {month}")

Year: 2011, Month: 1
Year: 2011, Month: 2
Year: 2011, Month: 3
Year: 2011, Month: 4
Year: 2011, Month: 5
Year: 2011, Month: 6
Year: 2011, Month: 7
Year: 2011, Month: 8
Year: 2011, Month: 9
Year: 2011, Month: 10
Year: 2011, Month: 11
Year: 2011, Month: 12
Year: 2012, Month: 1
Year: 2012, Month: 2
Year: 2012, Month: 3
Year: 2012, Month: 4
Year: 2012, Month: 5
Year: 2012, Month: 6
Year: 2012, Month: 7
Year: 2012, Month: 8
Year: 2012, Month: 9
Year: 2012, Month: 10
Year: 2012, Month: 11
Year: 2012, Month: 12
Year: 2013, Month: 1
Year: 2013, Month: 2
Year: 2013, Month: 3
Year: 2013, Month: 4
Year: 2013, Month: 5
Year: 2013, Month: 6
Year: 2013, Month: 7
Year: 2013, Month: 8
Year: 2013, Month: 9
Year: 2013, Month: 10
Year: 2013, Month: 11
Year: 2013, Month: 12
Year: 2014, Month: 1
Year: 2014, Month: 2
Year: 2014, Month: 3
Year: 2014, Month: 4
Year: 2014, Month: 5
Year: 2014, Month: 6
Year: 2014, Month: 7
Year: 2014, Month: 8
Year: 2014, Month: 9
Year: 2014, Month: 10
Year: 2014, Month: 11
Ye